## TLD

In [22]:
import os
from pathlib import Path
import pandas as pd
import geopandas as gpd
import numpy as np
from os.path import exists
import math
from geopy.distance import distance

# Current working directory
basepath = Path.cwd().parent.parent.parent

# Read list of stations within model coverage area
raw_data = os.path.join(basepath, '01 Raw Data' )
# raw_data

### Read Rail and Bus matrices

In [34]:
rail_mtx = pd.read_csv('weekday_prior_rail_matrices_10072024.csv')
rail_mtx.head()

,Unnamed: 0,org_zone,dest_zone,fare_peak,trips
0,0,101,1021,Fare-day,0.755
1,1,101,1021,No_Fare-day,0.037
2,2,101,1029,Fare-day,0.859
3,3,101,1030,Fare-day,1.279
4,4,101,1030,No_Fare-day,0.018


In [35]:
# Load zone  shapefile
# Load zone centroid shapefile
zone_gdf = gpd.read_file(f'{raw_data}/07 Connectors/v20/zone_centroids_zone_centroid.SHP').to_crs('epsg:4326')
zone_gdf.head()

,NO,MODEL_AREA,BUS_SUM,RAIL_SUM,geometry
0,101,External,0.0,85.22,POINT (-0.75482 53.11392)
1,201,External,0.0,178.37,POINT (0.51505 52.24452)
2,301,External,0.0,497.37,POINT (0.11834 51.17473)
3,401,External,0.0,50.82,POINT (-1.46530 54.34182)
4,505,External,0.0,221.27,POINT (-2.72097 54.05088)


In [36]:
rail_mtx_mapped = pd.merge(rail_mtx,zone_gdf[['NO','geometry','MODEL_AREA']],left_on='org_zone',right_on='NO', how='left')

rail_mtx_mapped['org_lat'] = rail_mtx_mapped['geometry'].apply(lambda p: p.y)
rail_mtx_mapped['org_lon'] = rail_mtx_mapped['geometry'].apply(lambda p: p.x)
rail_mtx_mapped.rename(columns={'MODEL_AREA':'org_model_area'}, inplace=True) 
rail_mtx_mapped.drop(columns=['NO','geometry'], inplace=True) 
rail_mtx_mapped = pd.merge(rail_mtx_mapped,zone_gdf[['NO','geometry']],left_on='dest_zone',right_on='NO', how='left')
rail_mtx_mapped['dest_lat'] = rail_mtx_mapped['geometry'].apply(lambda p: p.y)
rail_mtx_mapped['dest_lon'] = rail_mtx_mapped['geometry'].apply(lambda p: p.x)
rail_mtx_mapped.rename(columns={'MODEL_AREA':'dest_model_area'}, inplace=True) 
rail_mtx_mapped.drop(columns=['NO','geometry'], inplace=True) 
# Calculate the distance and assign it to the 'dist' column
rail_mtx_mapped['dist'] = rail_mtx_mapped.apply(lambda row: distance((row['org_lat'], row['org_lon']),
                                                                     (row['dest_lat'], row['dest_lon'])).miles,
                                                axis=1)


# Round the 'dist' column to 2 decimal places
rail_mtx_mapped['dist'] = rail_mtx_mapped['dist'].round(2)

rail_mtx_mapped.head()



,Unnamed: 0,org_zone,dest_zone,fare_peak,trips,org_model_area,org_lat,org_lon
0,0,101,1021,Fare-day,0.755,External,53.113922,-0.75482
1,1,101,1021,No_Fare-day,0.037,External,53.113922,-0.75482
2,2,101,1029,Fare-day,0.859,External,53.113922,-0.75482
3,3,101,1030,Fare-day,1.279,External,53.113922,-0.75482
4,4,101,1030,No_Fare-day,0.018,External,53.113922,-0.75482


,Unnamed: 0,org_zone,dest_zone,fare_peak,trips,org_model_area,org_lat,org_lon,dest_lat,dest_lon,dist
0,0,101,1021,Fare-day,0.755,External,53.113922,-0.75482,52.498213,-1.783836,60.60
1,1,101,1021,No_Fare-day,0.037,External,53.113922,-0.75482,52.498213,-1.783836,60.60
2,2,101,1029,Fare-day,0.859,External,53.113922,-0.75482,52.462499,-1.812137,63.19
3,3,101,1030,Fare-day,1.279,External,53.113922,-0.75482,52.478235,-1.738114,60.25
4,4,101,1030,No_Fare-day,0.018,External,53.113922,-0.75482,52.478235,-1.738114,60.25


In [39]:
rail_mtx_mapped.to_csv('rail_tld_10072024.csv')

In [40]:
bus_mtx = pd.read_csv('swift_csmt_od_27062024.csv')
bus_mtx.head()

,Unnamed: 0,org,dest,adj_trips,dseg
0,0,1029,13801,0.001389,Child_Student-Fare-AM
1,1,1029,13802,0.001736,Child_Student-Fare-AM
2,2,1029,13803,0.004514,Child_Student-Fare-AM
3,3,1029,13804,0.002083,Child_Student-Fare-AM
4,4,1029,13805,0.006250,Child_Student-Fare-AM


In [41]:
bus_mtx_mapped = pd.merge(bus_mtx,zone_gdf[['NO','geometry','MODEL_AREA']],left_on='org',right_on='NO', how='left')

bus_mtx_mapped['org_lat'] = bus_mtx_mapped['geometry'].apply(lambda p: p.y)
bus_mtx_mapped['org_lon'] = bus_mtx_mapped['geometry'].apply(lambda p: p.x)
bus_mtx_mapped.rename(columns={'MODEL_AREA':'org_model_area'}, inplace=True) 
bus_mtx_mapped.drop(columns=['NO','geometry'], inplace=True) 
bus_mtx_mapped = pd.merge(bus_mtx_mapped,zone_gdf[['NO','geometry']],left_on='dest',right_on='NO', how='left')
bus_mtx_mapped['dest_lat'] = bus_mtx_mapped['geometry'].apply(lambda p: p.y)
bus_mtx_mapped['dest_lon'] = bus_mtx_mapped['geometry'].apply(lambda p: p.x)
bus_mtx_mapped.rename(columns={'MODEL_AREA':'dest_model_area'}, inplace=True) 
bus_mtx_mapped.drop(columns=['NO','geometry'], inplace=True) 
# Calculate the distance and assign it to the 'dist' column
bus_mtx_mapped['dist'] = bus_mtx_mapped.apply(lambda row: distance((row['org_lat'], row['org_lon']),
                                                                     (row['dest_lat'], row['dest_lon'])).miles,
                                                axis=1)


# Round the 'dist' column to 2 decimal places
bus_mtx_mapped['dist'] = bus_mtx_mapped['dist'].round(2)

bus_mtx_mapped.head()

,Unnamed: 0,org,dest,adj_trips,dseg,org_model_area,org_lat,org_lon,dest_lat,dest_lon,dist
0,0,1029,13801,0.001389,Child_Student-Fare-AM,External,52.462499,-1.812137,52.411971,-1.509068,13.27
1,1,1029,13802,0.001736,Child_Student-Fare-AM,External,52.462499,-1.812137,52.410897,-1.505386,13.44
2,2,1029,13803,0.004514,Child_Student-Fare-AM,External,52.462499,-1.812137,52.412785,-1.499831,13.64
3,3,1029,13804,0.002083,Child_Student-Fare-AM,External,52.462499,-1.812137,52.413576,-1.504277,13.44
4,4,1029,13805,0.006250,Child_Student-Fare-AM,External,52.462499,-1.812137,52.415843,-1.501095,13.53


In [42]:
bus_mtx_mapped.to_csv('bus_tld_10072024.csv')